## Install AutoTrain

In [ ]:
!pip install -Uqq autotrain-advanced

## Export your Hugging Face credentials

In [ ]:
hf_token = "hf_XXX" #@param {type:"string"}
hf_username = "abc" #@param {type:"string"}

import os
os.environ["HF_TOKEN"] = hf_token
os.environ["HF_USERNAME"] = hf_username

## Run AutoTrain

In [ ]:
!git clone https://github.com/dnth/vl-hf-annotation-converter
!cd vl-hf-annotation-converter && autotrain --config configs/detr-resnet-50.yml

In [ ]:
!cd vl-hf-annotation-converter && python inference.py